In [1]:
import pandas as pd
import numpy as np
from scipy import signal
import math
import scipy.stats as stats
import glob
import warnings

import matplotlib.pyplot as plt
import glob
import os

from sklearn import svm, metrics, preprocessing
from sklearn.model_selection import cross_val_score

In [5]:
d=pd.read_csv('../../etc/step_num_UT/features.csv')

In [15]:
def diff(f, x):
    h=10e-10
    return (f(x+h)-f(x))/h

def func(x):
    return np.exp(kde([x]))

def make_line(x, a):
    d=diff(func, a)
    return d*(x-a)+func(a)

In [85]:
from scipy.stats import gaussian_kde
from sklearn.preprocessing import MinMaxScaler, minmax_scale
from sklearn.model_selection import KFold
from sklearn import svm, metrics, preprocessing
import warnings
warnings.simplefilter('ignore')
new_df=pd.DataFrame()
total_score=[]

for bw_m in [0.01, 0.05, 0.1, 0.5, 1.0, 5, 10]:
    for bw_h in [0.01, 0.05, 0.1, 0.5, 1.0, 5, 10]:
        for file in glob.glob("../../etc/label/*.csv"):
            file_name=os.path.basename(file).split(".csv")[0]
            SH=d[d['user']==file_name]['SC_H']
            SM=d[d['user']==file_name]['SC_M']
            labels=d[d['user']==file_name]['label'].values

            res_M=[]
            for i in range(0, len(SM)):
                res_M+=[i]*int(SM.values[i])
            kde=gaussian_kde(res_M, bw_method=bw_m)
            estimates_M=np.exp(kde(np.linspace(-1, len(SM)+1, num=len(SM)+2)))
            estimates_M=minmax_scale(estimates_M)
            gradient_M=minmax_scale([diff(func, i)[0] for i in range(len(SM))])

            res_H=[]
            for i in range(0, len(SH)):
                res_H+=[i]*int(SH.values[i])    
            kde=gaussian_kde(res_H, bw_method=bw_h)
            estimates_H=np.exp(kde(np.linspace(-1, len(SH)+1, num=len(SH)+2)))
            estimates_H=minmax_scale(estimates_H)
            gradient_H=minmax_scale([diff(func, i)[0] for i in range(len(SH))])

            count=0
            for i in range(1, len(SH)+1):
                new_df=new_df.append({"user":file_name, "label":labels[count], "SC_M":estimates_M[i], "SCP_M":estimates_M[i-1], "SCF_M":estimates_M[i+1],
                                      "SC_H":estimates_H[i], "SCP_H":estimates_H[i-1], "SCF_H":estimates_H[i+1], "gradient_M":gradient_M[count], "gradient_H":gradient_H[count]}, ignore_index=True)
                count+=1
        new_df.to_csv("../../etc/optimize/{}.csv".format('features'), index=False)
        new_df=pd.DataFrame()

        step_count=pd.read_csv('../../etc/optimize/features.csv')
        datas=pd.read_csv('../../etc/windows_features/clipped_std_features.csv')

        datas['SC_M']=step_count['SC_M']
        datas['SCF_M']=step_count['SCF_M']
        datas['SCP_M']=step_count['SCP_M']
        datas['gradient_M']=step_count['gradient_M']
        datas['SC_H']=step_count['SC_H']
        datas['SCF_H']=step_count['SCF_H']
        datas['SCP_H']=step_count['SCP_H']
        datas['gradient_H']=step_count['gradient_H']

        datas=datas[datas['user']!="7NM9zimotoB"]
        datas=datas[datas['user']!="7NM2zimotoD"]
        datas=datas[datas['user']!="6NM9yossi"]

        moving=datas[datas['label']=='m']
        observing=datas[datas['label']=='o']
        searching=datas[datas['label']=='s']

        moving['index']=0
        searching['index']=1
        observing['index']=2

        moving['indexMO']=0
        searching['indexMO']=0
        observing['indexMO']=1

        DATA=pd.concat([moving, searching, observing])

        kf=KFold(n_splits=10, shuffle=True, random_state=1)
        scores=[]

        for train_index, test_index in kf.split(DATA):
            df_train = DATA.iloc[train_index]
            df_test = DATA.iloc[test_index]

            x_train=df_train.drop(['label', 'user', 'index', 'indexMO'], axis=1)
            y_train=df_train[['index', 'indexMO']]
            x_test=df_test.drop(['label', 'user', 'index', 'indexMO'], axis=1)
            y_test=df_test[['index', 'indexMO']]

            #x_train=x_train[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
            #x_test=x_test[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
            x_train=x_train[['acc_std_window0', 'acc_x_std_window0', 'gyro_y_std_window0',
               'gyro_x_std_window1', 'acc_y_std_window2', 'acc_z_std_window2',
               'acc_kurtosis_window3', 'acc_skewness_window3', 'acc_std_window3',
               'acc_z_std_window3', 'SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]

            x_test=x_test[['acc_std_window0', 'acc_x_std_window0', 'gyro_y_std_window0',
               'gyro_x_std_window1', 'acc_y_std_window2', 'acc_z_std_window2',
               'acc_kurtosis_window3', 'acc_skewness_window3', 'acc_std_window3',
               'acc_z_std_window3', 'SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]

            clf = svm.SVC(kernel='rbf', gamma=0.6, C=0.6, class_weight='balanced')
            clf.fit(x_train, y_train[['indexMO']])
            clf_pr=clf.predict(x_test)

            y_test['pred_index']=clf_pr

            x2_test=x_test[y_test['pred_index']==0]
            y2_test=y_test[y_test['pred_index']==0][['index']]

            clf2 = svm.SVC(kernel='rbf', gamma=0.6, C=0.6, class_weight='balanced')
            x_train = x_train[(y_train['index']==0) | (y_train['index']==1)]
            #x_train = x_train[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
            y_train = y_train[(y_train['index']==0) | (y_train['index']==1)]
            clf2.fit(x_train, y_train[['index']])
            #clf2_pr=clf2.predict(x2_test[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']])
            clf2_pr=clf2.predict(x2_test)

            y2_test['pred2_index']=clf2_pr

            result=pd.DataFrame()
            result['result'] = y2_test['index']==y2_test['pred2_index']
            a=pd.DataFrame(y_test[y_test['pred_index']==1]['indexMO']==y_test[y_test['pred_index']==1]['pred_index'], columns=['result'])
            results=pd.concat([result, a])

            scores+=[len(results[results['result']==True])/len(results)]
        print("bw_m: {}, bw_h: {}, score: {}".format(bw_m, bw_h, sum(scores)/len(scores)))
        total_score+=[sum(scores)/len(scores)]

bw_m: 0.01, bw_h: 0.01, score: 0.70719212412645
bw_m: 0.01, bw_h: 0.05, score: 0.7087010826930793
bw_m: 0.01, bw_h: 0.1, score: 0.7104036445077879
bw_m: 0.01, bw_h: 0.5, score: 0.7178723254546653
bw_m: 0.01, bw_h: 1.0, score: 0.7142731793929016
bw_m: 0.01, bw_h: 5, score: 0.7129188998209485
bw_m: 0.01, bw_h: 10, score: 0.7106745872532054
bw_m: 0.05, bw_h: 0.01, score: 0.7132674515393045
bw_m: 0.05, bw_h: 0.05, score: 0.712918675257948
bw_m: 0.05, bw_h: 0.1, score: 0.7180655244893438
bw_m: 0.05, bw_h: 0.5, score: 0.7246055176626285
bw_m: 0.05, bw_h: 1.0, score: 0.7212771197250151
bw_m: 0.05, bw_h: 5, score: 0.7169817684785407
bw_m: 0.05, bw_h: 10, score: 0.7169431735842051
bw_m: 0.1, bw_h: 0.01, score: 0.7142733590433018
bw_m: 0.1, bw_h: 0.05, score: 0.71671088561659
bw_m: 0.1, bw_h: 0.1, score: 0.7217801408459139
bw_m: 0.1, bw_h: 0.5, score: 0.7282427795509938
bw_m: 0.1, bw_h: 1.0, score: 0.7251079549197262
bw_m: 0.1, bw_h: 5, score: 0.7212771796084818
bw_m: 0.1, bw_h: 10, score: 0.719

In [79]:
for file in glob.glob("../../etc/label/*.csv"):
    file_name=os.path.basename(file).split(".csv")[0]
    SH=d[d['user']==file_name]['SC_H']
    SM=d[d['user']==file_name]['SC_M']
    labels=d[d['user']==file_name]['label'].values

    res_M=[]
    for i in range(0, len(SM)):
        res_M+=[i]*int(SM.values[i])
    kde=gaussian_kde(res_M, bw_method=1.0)
    estimates_M=np.exp(kde(np.linspace(-1, len(SM)+1, num=len(SM)+2)))
    estimates_M=minmax_scale(estimates_M)
    gradient_M=minmax_scale([diff(func, i)[0] for i in range(len(SM))])

    res_H=[]
    for i in range(0, len(SH)):
        res_H+=[i]*int(SH.values[i])    
    kde=gaussian_kde(res_H, bw_method=0.5)
    estimates_H=np.exp(kde(np.linspace(-1, len(SH)+1, num=len(SH)+2)))
    estimates_H=minmax_scale(estimates_H)
    gradient_H=minmax_scale([diff(func, i)[0] for i in range(len(SH))])

    count=0
    for i in range(1, len(SH)+1):
        new_df=new_df.append({"user":file_name, "label":labels[count], "SC_M":estimates_M[i], "SCP_M":estimates_M[i-1], "SCF_M":estimates_M[i+1],
                              "SC_H":estimates_H[i], "SCP_H":estimates_H[i-1], "SCF_H":estimates_H[i+1], "gradient_M":gradient_M[count], "gradient_H":gradient_H[count]}, ignore_index=True)
        count+=1
new_df.to_csv("../../etc/optimize/{}.csv".format('features'), index=False)
new_df=pd.DataFrame()

step_count=pd.read_csv('../../etc/optimize/features.csv')
datas=pd.read_csv('../../etc/windows_features/clipped_std_features.csv')

datas['SC_M']=step_count['SC_M']
datas['SCF_M']=step_count['SCF_M']
datas['SCP_M']=step_count['SCP_M']
datas['gradient_M']=step_count['gradient_M']
datas['SC_H']=step_count['SC_H']
datas['SCF_H']=step_count['SCF_H']
datas['SCP_H']=step_count['SCP_H']
datas['gradient_H']=step_count['gradient_H']

datas=datas[datas['user']!="7NM9zimotoB"]
datas=datas[datas['user']!="7NM2zimotoD"]
datas=datas[datas['user']!="6NM9yossi"]

moving=datas[datas['label']=='m']
observing=datas[datas['label']=='o']
searching=datas[datas['label']=='s']

moving['index']=0
searching['index']=1
observing['index']=2

moving['indexMO']=0
searching['indexMO']=0
observing['indexMO']=1

DATA=pd.concat([moving, searching, observing])

kf=KFold(n_splits=10, shuffle=True, random_state=1)
#scores=[]
scores_m=[]
scores_s=[]
scores_o=[]

for train_index, test_index in kf.split(DATA):
    df_train = DATA.iloc[train_index]
    df_test = DATA.iloc[test_index]

    x_train=df_train.drop(['label', 'user', 'index', 'indexMO'], axis=1)
    y_train=df_train[['index', 'indexMO']]
    x_test=df_test.drop(['label', 'user', 'index', 'indexMO'], axis=1)
    y_test=df_test[['index', 'indexMO']]

    #x_train=x_train[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
    #x_test=x_test[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
    x_train=x_train[['acc_std_window0', 'acc_x_std_window0', 'gyro_y_std_window0',
       'gyro_x_std_window1', 'acc_y_std_window2', 'acc_z_std_window2',
       'acc_kurtosis_window3', 'acc_skewness_window3', 'acc_std_window3',
       'acc_z_std_window3', 'SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]

    x_test=x_test[['acc_std_window0', 'acc_x_std_window0', 'gyro_y_std_window0',
       'gyro_x_std_window1', 'acc_y_std_window2', 'acc_z_std_window2',
       'acc_kurtosis_window3', 'acc_skewness_window3', 'acc_std_window3',
       'acc_z_std_window3', 'SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]

    clf = svm.SVC(kernel='rbf', gamma=0.6, C=0.6, class_weight='balanced')
    clf.fit(x_train, y_train[['indexMO']])
    clf_pr=clf.predict(x_test)

    y_test['pred_index']=clf_pr

    x2_test=x_test[y_test['pred_index']==0]
    y2_test=y_test[y_test['pred_index']==0][['index']]

    clf2 = svm.SVC(kernel='rbf', gamma=0.6, C=0.6, class_weight='balanced')
    x_train = x_train[(y_train['index']==0) | (y_train['index']==1)]
    #x_train = x_train[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']]
    y_train = y_train[(y_train['index']==0) | (y_train['index']==1)]
    clf2.fit(x_train, y_train[['index']])
    #clf2_pr=clf2.predict(x2_test[['SC_M', 'SCF_M', 'SCP_M', 'gradient_M', 'SC_H', 'SCF_H', 'SCP_H', 'gradient_H']])
    clf2_pr=clf2.predict(x2_test)

    y2_test['pred2_index']=clf2_pr

    #result=pd.DataFrame()
    #result['result'] = y2_test['index']==y2_test['pred2_index']
    #a=pd.DataFrame(y_test[y_test['pred_index']==1]['indexMO']==y_test[y_test['pred_index']==1]['pred_index'], columns=['result'])
    #results=pd.concat([result, a])
    #scores+=[len(results[results['result']==True])/len(results)]
    
    """
    #Search精度確認用
    result_m=pd.DataFrame()
    y2_test_m=y2_test[y2_test['index']==0]
    result_m['result'] = y2_test_m['index']==y2_test_m['pred2_index']
    scores_m+=[len(result_m[result_m['result']==True])/len(df_test[df_test['index']==0])]
    
    result_s=pd.DataFrame()
    y2_test_s=y2_test[y2_test['index']==1]
    result_s['result'] = y2_test_s['index']==y2_test_s['pred2_index']
    scores_s+=[len(result_s[result_s['result']==True])/len(df_test[df_test['index']==1])]
    
    result_o=pd.DataFrame()
    y_test_o=y_test[y_test['index']==2]
    result_o['result'] = y_test_o['indexMO']==y_test_o['pred_index']
    scores_o+=[len(result_o[result_o['result']==True])/len(df_test[df_test['index']==2])]
    """
#print("bw_m: {}, bw_h: {}, score: {}".format(bw_m, bw_h, sum(scores)/len(scores)))
#total_score+=[sum(scores)/len(scores)]
print(sum(scores_m)/len(scores_m))
print(sum(scores_s)/len(scores_s))
print(sum(scores_o)/len(scores_o))

0.7334160866711319
0.5401482461460545
0.8931130255992089


In [81]:
print(len(result_m)+len(result_s)+len(result_o))

2294


In [82]:
len(y_test)

2584

In [78]:
len(a[a['result']==True])/len(a)

0.7058823529411765